# Exercise 3
The goal of the third exercise is to put the previous components together and demonstrate a typical usecase for `pyiron_base`.

# Create a dynamic job class
To interface with any kind of executable we can use the dynamic pyiron job classes.

Use the `resource_path` from exercise one and select the one which is not included in the your `conda` environment:

In [1]:
from pyiron_base import state
resource_path_selected = [p for p in state.settings.resource_paths if "conda" not in p][0]
resource_path_selected

'/Users/jan/pyiron/resources'

Create the directory `dynamic/MydynamicJob/bin` in your `resource_path_selected` if it does not exist already:

In [2]:
import os
os.makedirs(os.path.join(resource_path_selected, "dynamic", "MydynamicJob", "bin"), exist_ok=True)

In these directories we now create the following files:
* `dynamic/MydynamicJob/input.json` - default input 
* `dynamic/MydynamicJob/script.py` - the `write_input()` and `collect_output()` function to interface with files
* `dynamic/MydynamicJob/bin/run_mydynamicjob_0.0.1.sh` - the shell script to run

In [3]:
input_str = """{"a": 1, "b": [1, 2, 3]}"""
script_str = """\
import json
import os


def write_input(working_directory, input_dict):
    # json example
    with open(os.path.join(working_directory, "input.txt"), "w") as f:
        json.dump(input_dict, f)


def collect_output(working_directory):
    # json example
    with open(os.path.join(working_directory, "output.txt"), "r") as f:
        return json.load(f)
"""
run_str = """\
#!/bin/bash
number_of_cores=1
number_of_threads=1

# cat input.txt > output.txt
echo \"import json\nwith open('input.txt') as f:\n    i = json.load(f)\ni['a'] = i['a'] ** 2\nwith open('output.txt', 'w') as f:\n    json.dump(i, f)\" | python 
"""

In [4]:
file_name_lst = [
    os.path.join(resource_path_selected, "dynamic", "MydynamicJob", "input.json"),
    os.path.join(resource_path_selected, "dynamic", "MydynamicJob", "script.py"), 
    os.path.join(resource_path_selected, "dynamic", "MydynamicJob", "bin", "run_mydynamicjob_0.0.1.sh"),
]

In [5]:
for f, s in zip(file_name_lst, [input_str, script_str, run_str]):
    with open(f, "w") as fo:
        fo.writelines(s)

Set the executable bit for the shell script:

In [6]:
import subprocess
subprocess.check_output(
    ["chmod", "+x", os.path.join(resource_path_selected, "dynamic", "MydynamicJob", "bin", "run_mydynamicjob_0.0.1.sh")]
)

b''

# Parameter Study
Use the worker class to execute multiple jobs in one allocation. Start by creating a test project:

In [7]:
from pyiron_base import Project
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/11 [00:00<?, ?it/s]

Submit a series of jobs to the worker:

In [8]:
for i in range(10):
    job = pr.create.job.MydynamicJob("job_" + str(i))
    job.input["a"] = i
    job.run()

The job job_0 was saved and received the ID: 250
The job job_1 was saved and received the ID: 251
The job job_2 was saved and received the ID: 252
The job job_3 was saved and received the ID: 253
The job job_4 was saved and received the ID: 254
The job job_5 was saved and received the ID: 255
The job job_6 was saved and received the ID: 256
The job job_7 was saved and received the ID: 257
The job job_8 was saved and received the ID: 258
The job job_9 was saved and received the ID: 259


In [9]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,250,finished,None,job_0,/job_0,/Users/jan/pyiron/projects/,2022/2022-07-25-learn-pyiron-dev/test/,2022-07-26 00:30:59.886975,2022-07-26 00:30:59.960009,0.0,pyiron@MacMini.local#1,MydynamicJob,0.4,None,None
1,251,finished,None,job_1,/job_1,/Users/jan/pyiron/projects/,2022/2022-07-25-learn-pyiron-dev/test/,2022-07-26 00:31:00.002747,2022-07-26 00:31:00.068253,0.0,pyiron@MacMini.local#1,MydynamicJob,0.4,None,None
2,252,finished,None,job_2,/job_2,/Users/jan/pyiron/projects/,2022/2022-07-25-learn-pyiron-dev/test/,2022-07-26 00:31:00.113522,2022-07-26 00:31:00.178539,0.0,pyiron@MacMini.local#1,MydynamicJob,0.4,None,None
3,253,finished,None,job_3,/job_3,/Users/jan/pyiron/projects/,2022/2022-07-25-learn-pyiron-dev/test/,2022-07-26 00:31:00.220359,2022-07-26 00:31:00.285699,0.0,pyiron@MacMini.local#1,MydynamicJob,0.4,None,None
4,254,finished,None,job_4,/job_4,/Users/jan/pyiron/projects/,2022/2022-07-25-learn-pyiron-dev/test/,2022-07-26 00:31:00.329479,2022-07-26 00:31:00.393410,0.0,pyiron@MacMini.local#1,MydynamicJob,0.4,None,None
5,255,finished,None,job_5,/job_5,/Users/jan/pyiron/projects/,2022/2022-07-25-learn-pyiron-dev/test/,2022-07-26 00:31:00.434855,2022-07-26 00:31:00.500510,0.0,pyiron@MacMini.local#1,MydynamicJob,0.4,None,None
6,256,finished,None,job_6,/job_6,/Users/jan/pyiron/projects/,2022/2022-07-25-learn-pyiron-dev/test/,2022-07-26 00:31:00.544158,2022-07-26 00:31:00.608383,0.0,pyiron@MacMini.local#1,MydynamicJob,0.4,None,None
7,257,finished,None,job_7,/job_7,/Users/jan/pyiron/projects/,2022/2022-07-25-learn-pyiron-dev/test/,2022-07-26 00:31:00.649773,2022-07-26 00:31:00.714069,0.0,pyiron@MacMini.local#1,MydynamicJob,0.4,None,None
8,258,finished,None,job_8,/job_8,/Users/jan/pyiron/projects/,2022/2022-07-25-learn-pyiron-dev/test/,2022-07-26 00:31:00.757507,2022-07-26 00:31:00.821576,0.0,pyiron@MacMini.local#1,MydynamicJob,0.4,None,None
9,259,finished,None,job_9,/job_9,/Users/jan/pyiron/projects/,2022/2022-07-25-learn-pyiron-dev/test/,2022-07-26 00:31:00.862963,2022-07-26 00:31:00.926367,0.0,pyiron@MacMini.local#1,MydynamicJob,0.4,None,None


# collect data with pyiron_table
Define two functions to collect both entries of both the input and the output:

In [10]:
def get_a_in(job):
    return job["storage"]["input"]["a"]

In [11]:
def get_a_out(job):
    return job["storage"]["output"]["a"]

In [12]:
def get_b_in(job):
    return job["storage"]["input"]["b"]

In [13]:
def get_b_out(job):
    return job["storage"]["output"]["b"]

Create a `pyiron_table` object to iterate over all jobs in the project:

In [14]:
table = pr.create.table()
table.add["ain"] = get_a_in
table.add["bin"] = get_b_in
table.add["aout"] = get_a_out
table.add["bout"] = get_b_out
table.run()

The job table was saved and received the ID: 260


Loading and filtering jobs:   0%|          | 0/11 [00:00<?, ?it/s]

Processing jobs:   0%|          | 0/10 [00:00<?, ?it/s]

/Users/jan/PycharmProjects/pyiron_base/pyiron_base/jobs/datamining.py:620: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['bin', 'bout'], dtype='object')]

  self.pyiron_table._df.to_hdf(


Return the output as pandas dataframe:

In [15]:
table.get_dataframe()

,job_id,ain,bin,aout,bout
0,250,0,"[1, 2, 3]",0,"[1, 2, 3]"
1,251,1,"[1, 2, 3]",1,"[1, 2, 3]"
2,252,2,"[1, 2, 3]",4,"[1, 2, 3]"
3,253,3,"[1, 2, 3]",9,"[1, 2, 3]"
4,254,4,"[1, 2, 3]",16,"[1, 2, 3]"
5,255,5,"[1, 2, 3]",25,"[1, 2, 3]"
6,256,6,"[1, 2, 3]",36,"[1, 2, 3]"
7,257,7,"[1, 2, 3]",49,"[1, 2, 3]"
8,258,8,"[1, 2, 3]",64,"[1, 2, 3]"
9,259,9,"[1, 2, 3]",81,"[1, 2, 3]"


# Task
Update the shell script by replacing the line:
`cat input.txt > output.txt`
with the line: 
```
echo \"import json\nwith open('input.txt') as f:\n    i = json.load(f)\ni['a'] = i['a'] ** 2\nwith open('output.txt', 'w') as f:\n    json.dump(i, f)\" | python 
```
to calculate the squares of the input.